In [1]:
import pandas as pd
import numpy as np
import scipy.stats

# Непараметрические критерии
## Выживаемость
Давайте вернёмся к данным выживаемости пациентов с лейкоцитарной лимфомой из видео про критерий знаков:

$$49, 58, 75, 110, 112, 132, 151, 276, 281, 362^*$$

Измерено остаточное время жизни с момента начала наблюдения (в неделях); звёздочка обозначает цензурирование сверху — исследование длилось 7 лет, и остаточное время жизни одного пациента, который дожил до конца наблюдения, неизвестно.

Поскольку цензурировано только одно наблюдение, для проверки гипотезы $H0:medX=200$ на этих данных можно использовать критерий знаковых рангов — можно считать, что время дожития последнего пациента в точности равно 362, на ранг этого наблюдения это никак не повлияет.

Критерием знаковых рангов проверьте эту гипотезу против двусторонней альтернативы, введите достигаемый уровень значимости, округлённый до четырёх знаков после десятичной точки.

In [2]:
patients = np.array([49, 58, 75, 110, 112, 132, 151, 276, 281, 362])
median = 200
p_value = np.round(scipy.stats.wilcoxon(patients - median)[1], 4)
print('p_value', p_value)

p_value 0.2845


Достигаемый уровень значимости больше порога в 0.05 - значит нулевая гипотеза не отклоняется.

## Леса
В ходе исследования влияния лесозаготовки на биоразнообразие лесов острова Борнео собраны данные о количестве видов деревьев в 12 лесах, где вырубка не ведётся:

$$22,22,15,13,19,19,18,20,21,13,13,15$$

и в 9 лесах, где идёт вырубка:

$$17,18,18,15,12,4,14,15,10$$

Проверьте гипотезу о равенстве среднего количества видов в двух типах лесов против односторонней альтернативы о снижении биоразнообразия в вырубаемых лесах. Используйте ранговый критерий. Чему равен достигаемый уровень значимости? Округлите до четырёх знаков после десятичной точки.
### Ответ
Альтернатива должна быть 'greater', если мы сравниваем не вырубленные с леса с вырубленными.

In [3]:
virgin_forest = np.array([22,22,15,13,19,19,18,20,21,13,13,15])
not_virgin_forest = np.array([17,18,18,15,12,4,14,15,10])
p_value = np.round(scipy.stats.mannwhitneyu(virgin_forest, not_virgin_forest, alternative = 'greater')[1], 4)
print('p_value', p_value)

p_value 0.029


Достигаемый уровень значимости меньше порога в 0.05 - значит нулевая гипотеза отклоняется в пользу альтернативной.

## В космосе никто не услышит твой крик
28 января 1986 года космический шаттл "Челленджер" взорвался при взлёте. Семь астронавтов, находившихся на борту, погибли. В ходе расследования причин катастрофы основной версией была неполадка с резиновыми уплотнительными кольцами в соединении с ракетными ускорителями. Для 23 предшествовавших катастрофе полётов "Челленджера" известны температура воздуха и появление повреждений хотя бы у одного из уплотнительных колец.

С помощью бутстрепа постройте 95% доверительный интервал для разности средних температур воздуха при запусках, когда уплотнительные кольца повреждались, и запусках, когда повреждений не было. Чему равна его ближайшая к нулю граница? Округлите до четырёх знаков после запятой.

Чтобы получить в точности такой же доверительный интервал, как у нас:

- установите random seed = 0 перед первым вызовом функции get_bootstrap_samples, один раз
- сделайте по 1000 псевдовыборок из каждой выборки.

In [4]:
df = pd.read_csv("challenger.txt", delimiter = "\t")
df.head()

,Unnamed: 0,Temperature,Incident
0,Apr12.81,18.9,0
1,Nov12.81,21.1,1
2,Mar22.82,20.6,0
3,Nov11.82,20.0,0
4,Apr04.83,19.4,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 3 columns):
Unnamed: 0     23 non-null object
Temperature    23 non-null float64
Incident       23 non-null int64
dtypes: float64(1), int64(1), object(1)
memory usage: 632.0+ bytes


In [6]:
def get_bootstrap_samples(data, n_samples):
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples
def stat_intervals(stat, alpha):
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return boundaries

In [7]:
life_values = df[df.Incident == 0].Temperature.values
death_values = df[df.Incident == 1].Temperature.values

In [8]:
np.random.seed(0)

life_scores = np.array(list(map(np.mean, get_bootstrap_samples(life, 1000))))
death_scores = np.array(list(map(np.mean, get_bootstrap_samples(death, 1000))))
delta_scores = life_scores -  death_scores

print("95% confidence interval for the life temperature:",  stat_intervals(life_scores, 0.05))
print("95% confidence interval for the death temperature:",  stat_intervals(death_scores, 0.05))
print("95% confidence interval for the delta temperature:",  stat_intervals(delta_scores, 0.05))

95% confidence interval for the life temperature: [21.06875  23.575625]
95% confidence interval for the death temperature: [14.5125     20.71607143]
95% confidence interval for the delta temperature: [1.42299107 7.93861607]


## Гипотеза о равенстве температур
На данных предыдущей задачи проверьте гипотезу об одинаковой средней температуре воздуха в дни, когда уплотнительный кольца повреждались, и дни, когда повреждений не было. Используйте перестановочный критерий и двустороннюю альтернативу. Чему равен достигаемый уровень значимости? Округлите до четырёх знаков после десятичной точки.

Чтобы получить такое же значение, как мы:
- установите random seed = 0;
- возьмите 10000 перестановок.

In [9]:
def permutation_t_stat_ind(sample1, sample2):
    return np.mean(sample1) - np.mean(sample2)

In [10]:
def get_random_combinations(n1, n2, max_combinations):
    index = list(range(n1 + n2))
    indices = set([tuple(index)])
    for i in range(max_combinations - 1):
        np.random.shuffle(index)
        indices.add(tuple(index))
    return [(index[:n1], index[n1:]) for index in indices]

In [11]:
def permutation_zero_dist_ind(sample1, sample2, max_combinations = None):
    joined_sample = np.hstack((sample1, sample2))
    n1 = len(sample1)
    n = len(joined_sample)
    
    if max_combinations:
        indices = get_random_combinations(n1, len(sample2), max_combinations)
    else:
        indices = [(list(index), list(filter(lambda i: i not in index, range(n)))) \
                    for index in itertools.combinations(range(n), n1)]
    
    distr = [joined_sample[list(i[0])].mean() - joined_sample[list(i[1])].mean() \
             for i in indices]
    return distr

In [12]:
def permutation_test(sample, mean, max_permutations = None, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    t_stat = permutation_t_stat_ind(sample, mean)
    
    zero_distr = permutation_zero_dist_ind(sample, mean, max_permutations)
    
    if alternative == 'two-sided':
        return sum([1. if abs(x) >= abs(t_stat) else 0. for x in zero_distr]) / len(zero_distr)
    
    if alternative == 'less':
        return sum([1. if x <= t_stat else 0. for x in zero_distr]) / len(zero_distr)

    if alternative == 'greater':
        return sum([1. if x >= t_stat else 0. for x in zero_distr]) / len(zero_distr)

In [13]:
np.random.seed(0)
permutation_test(death_values, life_values, max_permutations = 10000)

0.0057

Достигаемый уровень значимости меньше порога в 0.05 - значит нулевая гипотеза отклоняется в пользу альтернативной.

## Результат
Был закреплен материал из лекций.